<a href="https://colab.research.google.com/github/Ryo-Tanohata/sig_beginner9/blob/main/beginner9_kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://signate.jp/competitions/434#abstract

#1 データの取得
自分の環境に合わせてデータを取得して読み込んで下さい。

In [ ]:
#signate_begginer9.zipをGoogle Drive からひっぱってくるやつ
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
 
auth.authenticate_user()
 
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate_begginer9.zip'", fields="files(id)").execute()
signate_api_key = results.get('files', [])
 
filename = "/content/input/signate_begginer9.zip"
os.makedirs(os.path.dirname(filename), exist_ok=True)
 
request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


#2 ファイルの解凍

In [ ]:
import os
path="./input"
zip_files=os.listdir(path)
zip_files

['signate_begginer9.zip']

In [ ]:
import zipfile
for i in zip_files:
    if ".zip" in i:
        with zipfile.ZipFile("input/"+i) as existing_zip:
            print(f'open {i}')
            existing_zip.extractall('input')
    else:
        continue

open signate_begginer9.zip


In [ ]:
train2=pd.read_csv('input/train.csv')
test2=pd.read_csv('input/test.csv')

In [ ]:
train2.head()

,id,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,target
0,0,56,1,5882,0,38,0,185315.0104,0.844417,134,1,0,10,1
1,3,50,0,1767,1,35,0,371620.3666,0.837448,136,0,0,187,1
2,7,41,0,250,1,30,0,372810.2084,0.819996,139,1,0,29,1
3,11,47,0,582,1,20,0,185780.6482,0.870314,137,0,0,43,0
4,12,60,0,80,0,25,0,265096.3999,0.828112,136,1,0,213,0


In [ ]:
test2.head()

,id,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,1,67,1,56,0,60,0,255844.0310,0.824480,140,0,0,147
1,2,58,0,320,0,38,0,163434.8513,5.281481,134,0,0,207
2,4,67,1,737,0,60,0,255278.8436,0.841388,140,0,0,113
3,5,48,0,231,1,60,0,500149.7955,0.742807,139,1,1,215
4,6,59,0,320,0,35,0,245489.2530,1.082861,132,1,0,215


# ライブラリのインストール

In [ ]:
import warnings
import gc
import pandas as pd
import pandas_profiling as pdp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, roc_curve, average_precision_score
warnings.simplefilter('ignore')

# データの前処理

In [ ]:
 # 読み込むデータが格納されたディレクトリのパス，必要に応じて変更の必要あり
path = "/content/input/"

train_df = pd.read_csv(path + "train.csv")
test_df = pd.read_csv(path + "test.csv")
submit = pd.read_csv(path + "sample_submission.csv", header=None)

# 訓練データ、テストデータがわかるようにダミーの目的変数を代入
test_df['target']=-999

# 訓練データ、テストデータを結合
all_df = pd.concat([train_df,test_df])
del train_df,test_df
gc.collect()

# def eGFR(x):
#     if x['sex']==1:
#       return 194*(x['serum_creatinine']**-1.094)*(x['age']**-0.287)
#     else:
#       return 194*(x['serum_creatinine']**-1.094)*(x['age']**-0.287)*0.739

# all_df["eGFR"] = all_df.apply(eGFR, axis=1)

# 訓練データ、テストデータの分割
train_df = all_df[all_df['target']!=-999]
test_df = all_df[all_df['target']==-999]

y=train_df['target']
X=train_df.drop(['target','id'], axis=1)
test=test_df.drop(['target','id'], axis=1)

## 交差検証の準備

In [ ]:
folds = 5
kf = StratifiedKFold(n_splits=folds,shuffle=True,random_state=0)

In [ ]:
feats=list(data.columns)
feats

['age',
 'anaemia',
 'creatinine_phosphokinase',
 'diabetes',
 'ejection_fraction',
 'high_blood_pressure',
 'platelets',
 'serum_creatinine',
 'serum_sodium',
 'sex',
 'smoking',
 'time']

# 学習・予測

In [ ]:
preds={} #アンサンブル前の予測

## HistGradientBoostingClassifier

### Kfold

In [ ]:
y_cv_train_pred=y.copy()
y_cv_pred=y.copy()
y_test_list=[]
for i,(train_index,valid_index) in enumerate(kf.split(X, y)):
    X_cv_train = X.iloc[train_index]
    X_cv_valid = X.iloc[valid_index]
    y_cv_train = y.iloc[train_index]
    y_cv_valid = y.iloc[valid_index]
 
    model = HistGradientBoostingClassifier(loss='binary_crossentropy',max_iter=10001,learning_rate=0.1,random_state=0)
    model.fit(X_cv_train,y_cv_train)
    y_cv_train_pred[train_index] = model.predict(X_cv_train)
    y_cv_pred[valid_index] = model.predict(X_cv_valid) #train時のデータに合わせてpredを入れる #本コンペの評価はaccuracyのため確率のみ算出
    y_test_preds = model.predict_proba(test)
    y_test_list.append(y_test_preds)
    print(f'Fold {i+1}, ACC : {accuracy_score(y_cv_train, y_cv_train_pred[train_index]):.6f}, val_ACC : {accuracy_score(y_cv_valid, y_cv_pred[valid_index]):.6f}')
    #y_cv_predproba_list.append(model.predict_proba(test_))
 
print('Full ACC score %.6f' % accuracy_score(y, y_cv_pred))

Fold 1, ACC : 1.000000, val_ACC : 0.870000
Fold 2, ACC : 1.000000, val_ACC : 0.830000
Fold 3, ACC : 1.000000, val_ACC : 0.870000
Fold 4, ACC : 1.000000, val_ACC : 0.890000
Fold 5, ACC : 1.000000, val_ACC : 0.855000
Full ACC score 0.863000


In [ ]:
pred_proba_hist=0
for i in y_test_list:
    pred_proba_hist += i
pred_proba=pred_proba_hist/len(y_test_list)
pred={"hist": pred_proba} #1000個の推論データ
preds.update(pred)
#test_pred=(pred_proba_hist/len(y_test_list)).argmax(axis=1)
#test_pred

## LightGBMClassifier

In [ ]:
import lightgbm as lgb
y_cv_train_pred=y.copy()
y_cv_pred=y.copy()
y_test_list=[]
for i,(train_index,valid_index) in enumerate(kf.split(X, y)):
    X_cv_train = X.iloc[train_index]
    X_cv_valid = X.iloc[valid_index]
    y_cv_train = y.iloc[train_index]
    y_cv_valid = y.iloc[valid_index]
 
    lgb_train = lgb.Dataset(X_cv_train, y_cv_train,)
    lgb_eval = lgb.Dataset(X_cv_valid, y_cv_valid, reference=lgb_train,)

    model = lgb.LGBMClassifier(loss='binary_crossentropy',max_iter=10001,learning_rate=0.1,random_state=0)
    model.fit(X_cv_train, y_cv_train, eval_set=[(X_cv_valid, y_cv_valid)],
              verbose=False, eval_metric='accuracy')#, categorical_feature=categorical_features) # モデルの学習
    y_cv_train_pred[train_index] = model.predict(X_cv_train)
    y_cv_pred[valid_index] = model.predict(X_cv_valid) #train時のデータに合わせてpredを入れる #本コンペの評価はaccuracyのため確率のみ算出
    y_test_preds = model.predict_proba(test)
    y_test_list.append(y_test_preds)
    print(f'Fold {i+1}, ACC : {accuracy_score(y_cv_train, y_cv_train_pred[train_index]):.6f}, val_ACC : {accuracy_score(y_cv_valid, y_cv_pred[valid_index]):.6f}')
    #y_cv_predproba_list.append(model.predict_proba(test_))
 
print('Full ACC score %.6f' % accuracy_score(y, y_cv_pred))

Fold 1, ACC : 1.000000, val_ACC : 0.870000
Fold 2, ACC : 1.000000, val_ACC : 0.835000
Fold 3, ACC : 1.000000, val_ACC : 0.860000
Fold 4, ACC : 1.000000, val_ACC : 0.885000
Fold 5, ACC : 1.000000, val_ACC : 0.835000
Full ACC score 0.857000


In [ ]:
pred_proba_gbm=0
for i in y_test_list:
    pred_proba_gbm += i
pred_proba=pred_proba_gbm/len(y_test_list)
pred={"lgbm": pred_proba} #1000個の推論データ
preds.update(pred)

In [ ]:
sub_pred_proba=(preds['hist']+preds['lgbm'])/2
sub_pred=sub_pred_proba.argmax(axis=1)
sub_pred

In [ ]:
submit[1] = sub_pred
submit.to_csv(path + "submission.csv", index=False, header=False)

In [ ]:
pd.read_csv(path + "submission.csv", header=None)

,0,1
0,1,0
1,2,1
2,4,0
3,5,0
4,6,0
...,...,...
995,1992,0
996,1993,0
997,1995,0
998,1997,0
